In [1]:
import numpy as np
import torch
import pychop as pychop
from pychop.chop import chop
from pychop import float_params
from time import time
from numpy import linalg
import jax
# from pychop.chop import chop
# from pychop.quant import quant
from time import time
from scipy.io import savemat
np.set_printoptions(precision=4)

In [2]:
np.random.seed(0)

X_np = np.random.randn(5000, 5000) # Numpy array
X_th = torch.Tensor(X_np) # torch array
X_jx = jax.numpy.asarray(X_np)
print(X_np)

savemat("tests/verify.mat", {"X":X_np})

[[ 1.7641  0.4002  0.9787 ...  0.9292  0.2294  0.4144]
 [ 0.3097 -0.7375 -1.5369 ...  0.5169 -0.0329  1.2981]
 [-0.2021 -0.8332  1.7336 ...  0.7531 -0.581  -0.1984]
 ...
 [ 1.0743  1.1885  0.5093 ...  0.0705  0.5998 -2.4103]
 [ 0.3243 -0.0234  1.6287 ... -0.1609 -1.5977  1.4147]
 [ 0.6346  1.3809  0.5483 ...  0.3076 -0.1108  0.8386]]


### print unit-roundoff in machine

In [3]:
float_params()

,,u,xmins,xmin,xmax,p,emins,emin,emax
0,q43,6.25e-02,1.95e-03,1.56e-02,2.40e+02,4,-9,-6,7
1,q52,1.25e-01,1.53e-05,6.10e-05,5.73e+04,3,-16,-14,15
2,b,3.91e-03,9.18e-41,1.18e-38,3.39e+38,8,-133,-126,127
3,h,4.88e-04,5.96e-08,6.10e-05,6.55e+04,11,-24,-14,15
4,t,4.88e-04,1.15e-41,1.18e-38,3.40e+38,11,-136,-126,127
5,s,5.96e-08,1.40e-45,1.18e-38,3.40e+38,24,-149,-126,127
6,d,1.11e-16,4.94e-324,2.23e-308,1.80e+308,53,-1074,-1022,1023
7,q,9.63e-35,0.00e+00,0.00e+00,inf,113,-16494,-16382,16383


### set backend

In [4]:
# pychop.backend('torch')
pychop.backend('numpy', 1) # print information, NumPy is the default option.

Load NumPy backend.


### run chop

In [5]:
pyq_f = chop('h')
st = time()
X_bit = pyq_f(X_np)
print("runtime:", time() - st)
print(X_bit[:10, :5])

runtime: 2.01943302154541
[[ 1.7637e+00  4.0015e-01  9.7852e-01  2.2402e+00  1.8672e+00]
 [ 3.0981e-01 -7.3730e-01 -1.5371e+00 -5.6201e-01 -1.5996e+00]
 [-2.0215e-01 -8.3301e-01  1.7334e+00  1.9067e-01 -1.7786e-01]
 [ 2.4707e+00  5.8691e-01 -3.9746e-01  4.3359e-01  5.0049e-01]
 [ 3.3008e-01 -4.7994e-04  8.1836e-01  4.2822e-01 -2.5039e+00]
 [ 3.0200e-01  1.4673e-01  6.1035e-01  1.5518e+00  1.2192e-02]
 [ 3.7134e-01  3.0469e-01  5.0391e-01  1.3525e-01  6.5381e-01]
 [ 3.8721e-01  1.3926e+00 -8.5254e-01 -1.9922e-01 -3.2568e-01]
 [-1.9395e+00  7.3877e-01  1.3945e+00 -8.1348e-01 -8.1885e-01]
 [ 5.6152e-01 -1.3281e+00  4.4952e-02  1.7764e+00  3.6230e-01]]


In [3]:
pychop.backend('torch', 1) # print information
pyq_f = chop('h')
st = time()
X_bit = pyq_f(X_th)
print("runtime:", time() - st)
print(X_bit)

Load Troch backend.


NameError: name '_chop_round_towards_plus_inf' is not defined

In [5]:
pychop.backend('jax', 1) # print information
pyq_f = chop('h')
st = time()
X_bit = pyq_f(X_jx)
print("runtime:", time() - st)
print(X_bit)

Load JAX backend.
runtime: 5.955718040466309
[[ 1.7637  0.4001  0.9785 ...  0.9292  0.2294  0.4143]
 [ 0.3098 -0.7373 -1.5371 ...  0.5171 -0.0329  1.2979]
 [-0.2021 -0.833   1.7334 ...  0.7529 -0.5811 -0.1984]
 ...
 [ 1.0742  1.1885  0.5093 ...  0.0706  0.5996 -2.4102]
 [ 0.3242 -0.0234  1.6289 ... -0.1609 -1.5977  1.415 ]
 [ 0.6348  1.3809  0.5483 ...  0.3076 -0.1108  0.8384]]


In [3]:
pychop.backend('jax', 1) # print information
pyq_f = chop('h')
st = time()
X_bit = pyq_f(X_jx)
print("runtime:", time() - st)
print(X_bit)

Load JAX backend.


NonConcreteBooleanIndexError: Array boolean indices must be concrete; got ShapedArray(bool[5000,5000])

See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.NonConcreteBooleanIndexError

Load JAX backend.


NonConcreteBooleanIndexError: Array boolean indices must be concrete; got ShapedArray(bool[5000,5000])

See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.NonConcreteBooleanIndexError

### integer quantization

In [8]:
pychop.backend('numpy')
pyq_f = pychop.quant(bits=8)
X_q = pyq_f(X_np)
X_inv = pyq_f.dequant(X_q)
linalg.norm(X_inv - X_np)

62.800703887228494

In [9]:
pychop.backend('torch')
pyq_f = pychop.quant(bits=8)
X_q = pyq_f(X_th)
X_inv = pyq_f.dequant(X_q)
linalg.norm(X_inv - X_np)

62.800703841249906

In [10]:

pychop.backend('jax')
pyq_f = pychop.quant(bits=8)
X_q = pyq_f(X_jx)
X_inv = pyq_f.dequant(X_q)
linalg.norm(X_inv - X_jx)

62.7823

### fixed point quantization

In [11]:
pychop.backend('numpy')
pyq_f = pychop.fpoint()

pyq_f(X_np)

array([[ 1.75  ,  0.375 ,  1.    , ...,  0.9375,  0.25  ,  0.4375],
       [ 0.3125, -0.75  , -1.5625, ...,  0.5   , -0.0625,  1.3125],
       [-0.1875, -0.8125,  1.75  , ...,  0.75  , -0.5625, -0.1875],
       ...,
       [ 1.0625,  1.1875,  0.5   , ...,  0.0625,  0.625 , -2.4375],
       [ 0.3125, -0.    ,  1.625 , ..., -0.1875, -1.625 ,  1.4375],
       [ 0.625 ,  1.375 ,  0.5625, ...,  0.3125, -0.125 ,  0.8125]])

In [12]:
pychop.backend('torch')
pyq_f = pychop.fpoint()
pyq_f(X_th)

tensor([[ 1.7500,  0.3750,  1.0000,  ...,  0.9375,  0.2500,  0.4375],
        [ 0.3125, -0.7500, -1.5625,  ...,  0.5000, -0.0625,  1.3125],
        [-0.1875, -0.8125,  1.7500,  ...,  0.7500, -0.5625, -0.1875],
        ...,
        [ 1.0625,  1.1875,  0.5000,  ...,  0.0625,  0.6250, -2.4375],
        [ 0.3125, -0.0000,  1.6250,  ..., -0.1875, -1.6250,  1.4375],
        [ 0.6250,  1.3750,  0.5625,  ...,  0.3125, -0.1250,  0.8125]])

In [13]:
pychop.backend('jax')
pyq_f = pychop.fpoint()
pyq_f(X_jx)

Array([[ 1.75  ,  0.375 ,  1.    , ...,  0.9375,  0.25  ,  0.4375],
       [ 0.3125, -0.75  , -1.5625, ...,  0.5   , -0.0625,  1.3125],
       [-0.1875, -0.8125,  1.75  , ...,  0.75  , -0.5625, -0.1875],
       ...,
       [ 1.0625,  1.1875,  0.5   , ...,  0.0625,  0.625 , -2.4375],
       [ 0.3125, -0.    ,  1.625 , ..., -0.1875, -1.625 ,  1.4375],
       [ 0.625 ,  1.375 ,  0.5625, ...,  0.3125, -0.125 ,  0.8125]],      dtype=float32)